# **ch.09 텍스트를 위한 인공 신경망**  
## 09-1 순차 데이터와 순환 신경망  
### 풀어야 할 문제:  
> ### sequential data와 recurrent neural network에 대해 알아보기  

<br/>

### *순차 데이터*  
sequential data
+ 순서에 의미가 있는 data
+ 종류
  + text data
  + time series data  
  ex) 일별 기온 data

<br/>

지금까지 다루었던 data는 순서가 중요하지 않았음  
Fashion MNIST의 경우 sample의 순서가 model의 performance에 영향을 주지 않았음  
오히려 골고루 sample을 섞어야 더 나은 결과를 얻을 수 있었음


그러나 이제부터 다룰 text data는 단어의 순서가 중요한 sequential data  
sequential data는 순서를 유지하며 neural network에 입력해야 함  
&rarr; sequential data를 다룰 때는 입력한 data를 기억하는 기능이 필요함

<br/>

fully-connected neural network나 convolution neural network는 기억 장치가 없음  
하나의 smaple(or batch)를 사용하여 연산을 수행하고 나면 해당 sample(or batch)는 버려지고 다음 sample을 처리할 때 재사용하지 않음  
&rarr; 이렇게 input data의 flow가 앞으로만 전달되는 neural network를 feedfoward neural network (FFNN)이라고 함  
$\therefore$ 지금까지 다루었던 fully-connected neural network나 convolution neural network는 FFNN


neural network가 이전에 처리했던 sample을 다음 sample을 처리하는데 재사용하기 위해서는 이렇게 data flow가 앞으로만 전달되어서는 안됨  
&rarr; 다음 sample을 위해 이전 data가 layer에 순환될 필요가 있음

<br/>  
- - -
<br/>  

### *순환 신경망*  
recurrent neural network (RNN):  
fully-connected neural network와 거의 비슷하지만, 이전 data의 processing flow를 순환하는 고리를 추가한 것


RNN속 neuron은 sample을 처리할 때 바로 이전에 사용했던 data를 재사용함  

<br/>

예를 들어 $A, B, C$ 3개의 sample을 처리하는 RNN이 있다고 할 때, neuron의 출력을 각각 $O_A, O_B, O_C$라고 함  
첫 번째 sample $A$를 처리하고 나면 $O_A$가 나오는데, 이 output이 다시 neuron으로 들어가 $B$를 처리하는데 사용됨  
$O_A$에는 $A$의 information이 담겨있음  
같은 방식으로 $B$와 $C$를 처리함  
$C$를 처리할 때는 $O_B$가 함께 사용되고, $O_B$에는 $B$의 information이 더 많이 들어있겠지만, $A$의 information도 포함되어 있음  
&rarr; $O_C$에는 $A, B, C$의 information이 모두 들어있음  
&rarr; RNN은 이전 sample에 대한 기억을 가지고 있다고 표현함

<br/>

위 예시에서와 같이 sample을 처리하는 한 단계를 timestep이라고 함  
RNN은 이전 timestep의 sample을 기억하지만 timestep이 오래될수록 순환되는 information은 희미해짐


RNN에서는 layer를 cell이라고 함  
한 cell에는 여러 개의 neuron이 있지만, fully-connected neural network와 달리 neuron을 모두 표시하지 않고 하나의 cell로 layer를 표현함  
또 cell의 output을 hidden state라고 함

<br/>

일반적으로 RNN의 hidden layer의 activation function으로는 hyperbolic tangent function (tanh)이 많이 사용됨  
tanh도 'S' 모양이기에 sigmoid function으로 부르기도 함  
tanh은 -1~1사이의 범위를 가짐

<br/>

feedfoward neural network에서 neuron은 inputs과 weights를 곱함  
RNN에서도 동일하지만, RNN의 neuron에는 한 가지 weight가 더 있음  
&rarr; 이전 timestep의 hidden state에 곱해지는 weight  
&rarr; cell은 input과 이전 timestep의 hidden state를 사용하여 현재 timestep의 hidden state를 만듦


모든 timestep에서 사용되는 weight은 하나임  
이 weight은 timestep에 따라 변화되는 neuron의 output을 학습함  
&rarr; 이러한 기능이 sequential data를 다루는 데 필요함

<br/>  
- - -  
<br/>  

### *셀의 가중치와 입출력*  
RNN의 cell에서 필요한 weights shape 게산  
입력되는 feature 수: 4, cell의 neuron 수: 3인 상황  
+ intput이 가장 처음 만나는 weights:  
input layer와 cell의 neuron이 모두 fully-connect되므로 $4 \times 3 = 12$
+ 재사용되는 hidden state를 위한 weights:  
cell에 있는 각 neuron의 hidden state가 다음 timestep에 사용될 때 해당 neuron을 비롯한 모든 3개의 neuron에 전달되므로 $3 \times 3 = 9$
+ model parameter 수:  
모든 weights와 neuron당 하나의 intercepts까지 $12 + 9 + 3 = 24$


&rarr; hidden state가 모든 neuron에 순환되기 때문에 fully-connected neural network처럼 표현하기에는 무리가 있음

<br/>

cell의 input과 output
+ cell은 일반적으로 sample(sequence)마다 2개의 dimension을 가짐  
sequence length와 단어 표현 dimension을 가짐
+ sequence 안에는 여러 item이 있는데, 이 중 sequence의 length가 timestep의 length  
예를 들어 어떤 sequence에 "I am a boy"라는 문장이 있고, 각 단어를 3개의 어떠한 number로 표현한다고 할 때, 이 sequence의 shape은 (1, 4, 3)임
+ sequence가 cell을 통과하면 두 번째, 세 번째 dimension이 사라지고 cell의 neuron 개수만큼의 크기를 가진 1 dimension array를 출력함

<br/>

앞에서는 cell이 모든 timestep에서 output을 만들어 hidden state를 만드는 것처럼 표현했지만, 사실 cell은 기본적으로 마지막 timestep의 hidden state만 출력함  
&rarr; 이는 마치 입력된 sequence length를 모두 읽어 information을 마지막 hidden state에 압축하여 전달하는 것처럼 볼 수 있음  
&rarr; RNN이 sequential data를 잘 처리하는 이유를 파악할 수 있음

<br/>

RNN도 여러 개의 layer를 쌓을 수 있음  


cell의 input마다 timestep과 단어 표현으로 이루어진 2 dimension array이어야 함  
&rarr; cell이 여러 개 쌓인 RNN에서는 첫 번째 cell이 마지막 timestep의 hidden state만 출력해서는 안됨  
$\therefore$ cell을 여러 개 쌓은 RNN에서는 마지막 cell을 제외한 다른 모든 cell은 모든 timestep의 hidden state를 출력함


output layer에는 dense layer를 두어 class를 분류함  
마지막 cell의 hidden state가 1 dimension array이므로 flatten layer를 추가하여 평탄화할 필요가 없음  
multi-class classification일 경우에는 output layer에 class 개수만큼 neuron을 두고 softmax activation function 사용  
binary classification일 경우에는 하나의 neuron을 두고 sigmoid activation function 사용